## Keras & Deep Learning
- book : [Deep Learning with Python by Francois Chollet](https://www.amazon.com/Deep-Learning-Python-Francois-Chollet/dp/1617294438)
- source code : [github](https://github.com/fchollet/deep-learning-with-python-notebooks)
- CogPsi #2 2019.3.26. Yoon Kyung Lee
- [git repository and notebooks](https://github.com/yoonlee78/cogpsi)

<font color = blue> 3.6. 주택 가격 예측: 회귀 문제 </font> 
-----
### Boston Housing Price dataset

<img src = "./imgs/backbay.jpg" width = 800>

### 차례

3.6.1. 보스턴 주택 가격 데이터셋 <br>
3.6.2. 데이터 준비 <br>
3.6.3. 모델 구성 <br>
3.6.4. K-겹 검증을 사용한 훈련 검증 <br>
3.6.5. 정리


### 3.6.1. 보스턴 주택 가격 데이터셋 

1970년 중반 보스턴 외곽 지역의 범죄율, 지방세율 등의 데이터가 주어졌을 때 주택 가격의 중간 값을 예측해보자. 

이 데이터셋은 이전 2개(IMDB, Reuters)와 다르다. 데이터 포인트가 **506**개로 비교적 개수가 **적고** 404개는 훈련 샘플로, 102개는 테스트 샘플로 나뉘어져있다. 

입력 데이터에 있는 각 **특성(feature)**(예를 들어 범죄율)은 스케일이 서로 다르다. 어떤 값은 0과 1사이의 비율을 나타내고, 어떤 것은 1과 12사이의 값을 가지거나 1과 100 사이의 값을 가진다. 

In [ ]:
from keras import models, layers
from keras.datasets import boston_housing#데이터셋 로드하기
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

훈련 샘플과 테스트 샘플 확인

In [ ]:
train_data.shape 

In [ ]:
test_data.shape

404개의 훈련 샘플과 102개의 테스트 샘플이 있고 모두 13개의 수치 특성이 있다. 이 특성들은 1인당 범죄율, 주택당 평균 방의 개수, 고속도로 접근성 등이다. 

(책에 없음)
Feature 목록 

- Per capita crime rate.
- Proportion of residential land zoned for lots over 25,000 square feet.
- Proportion of non-retail business acres per town.
- Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
- Nitric oxides concentration (parts per 10 million).
- Average number of rooms per dwelling.
- Proportion of owner-occupied units built prior to 1940.
- Weighted distances to five Boston employment centres.
- Index of accessibility to radial highways.
- Full-value property-tax rate per $10,000.
- Pupil-teacher ratio by town.
- 1000 * (Bk - 0.63) ** 2 where Bk is the proportion of Black people by town.
- % lower status of the population.


타겟은 **주택 가격의 중간값(median values of owner-occupied homes)***으로 천 달러 단위이다. 

일반적으로 1만 달러에서 5만 달러 사이이다. 1970년대 중반 시점의 집세라는 점을 감안해야하고 아직 인프레이션에 영향을 받지 않은 가격이다. ("_The prices are typically between \$10,000 and \$50,000. If that sounds cheap, remember this was the mid-1970s, and these prices are not inflation-adjusted._")


<font color = blue> Comment : *책 번역은 <strong>'중간 가격'</strong>이라고 되어있으며 이 가격이 의미하는 바에 대한 설명 또한 번역을 거치고 나니 잘 전달이 되지 않는 느낌이 든다. 이 데이터의 설명은 미국의 역사/문화적인 부분이 반영된 만큼 오해의 소지가 있을 수 있으므로 원 저자의 코드와 설명 확인을 권장한다. </font> 

In [ ]:
train_targets

### 3.6.1. 데이터 준비

범위가 많이 다른 ("wildly different") 값을 신경망에 주입하면 문제가 된다. 특성 스케일이 다르면 global minimum을 찾아가는 gradient descent의 경로가 스케일이 큰 특성에 영향을 많이 받고, 학습을 더 어렵게 만듦. 그래서 정규화(standardization,표준화라고도 함)가 필요함. 

입력 데이터에 있는 각 특성(입력 데이터 행렬의 열)에 대해서 특성을 평균을 빼고 표준 편차로 나눈다(특성의 중앙이 0 근처에 맞추어지고 표준 편차가 1이 되게함). 

넘파이로 하는 정규화는 다음과 같다.  

In [ ]:
#데이터 정규화하기 

mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

여기서 테스트 데이터(test_data)를 정규화할 때 사용한 값이 훈련 데이터에서 계산한 값임을 주목해야한다. 머신 러닝 작업 과정에서 절대로 테스트 데이터에서 계산한 어떤 값도 사용해서는 안 된다. 데이터 정규화처럼 간단한 작업조차도 그렇다. **쉽게 생각하면, 훈련 데이터와 테스트 데이터를 각각 다른 스케일로 변환하게 되면 훈련 데이터에서 학습한 정보가 쓸모없게 되는 셈이다. 마찬가지로 실전에 투입하여 새로운 데이터에 대한 예측을 할 때도 훈련 데이터에서 계산한 값을 사용하여 정규화해야한다.**

### 3.6.3 모델 구성

샘플 개수가 적기 때문에 64 유닛을 가진 2개의 은닉 층으로 작은 네트워크를 구성하여 사용하고자 한다. 

일반적으로 훈련 데이터의 개수가 적을수록 과대적합(overfitting)이 더 쉽게 일어나므로 작은 모델을 사용하는 것이 과대적합을 피하는 한 방법이다. 

In [ ]:
def build_model():
#동일한 모델을 여러 번 생성할 것이므로 함수를 만들어 사용하자
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

이 네트워크의 마지막 층은 하나의 유닛을 가지고 있고 활성화 함수가 없다 (선형 층이라고 부른다 = 전형적인 스칼라 회귀 (_하나의 연속적인 값을 예측하는 회귀_)를 위한 구성. 

활성화 함수를 적용하면 출력 값의 범위를 제한하는 것 (예: sigmoid = 0~1 사이의 값 예측)

여기서는 마지막 층이 순수한 선형이므로 네트워크가 어떤 범위의 (연속적인) 값이라도 예측하도록 자유롭게 학습된다. 

따라서, 이 모델은 ```mse```손실 함수를 사용하여 컴파일한다. 이 함수는 **평균 제곱 오차(mean squared error)**의 약어로 **예측과 타겟 사이의 거리의 제곱**이다. (회귀 문제에서 기본적이고 널리 사용되는 손실 함수다). 

또한, 훈련하는 동안 모니터링을 위해 **평균 절대 오차(Mean Absolute Error, MAE)를 측정할 것이다.** 이는 예측과 타겟 사이 거리의 **절대값**이다. 예를 들어, 이 예제에서  MAE가 0.5면 예측이 평균적으로 500달러 정도 차이가 난다는 뜻이다. 

### 3.6.4 K-겹 검증을 사용한 훈련 검증

매개변수(e.g., num_epochs for training)들을 조정하면서 모델을 평가하기 위해 이전 예제처럼 데이터를 훈련 세트와 검증 세트로 나눈다. 

데이터 포인트가 많지 않기 때문에 검증 세트도 따라서 작아진다 (약 100개의 샘플). 

검증 세트와 훈련 세트로 어떤 데이터 포인트가 선택되었는지에 따라 검증 점수가 크게 달라진다. 검증 세트의 분할(validation split)에 대한 검증 점수의 분산(variance)이 높기 때문에 신뢰 있는 모델 평가를 하기 어렵게 된다. 

이럴때 적합한 방법은 <font color = blue> K-겹 교차 검증(K-fold cross-validation)</font>을 사용하는 것이다. 

데이터를 K개의 분할 (=fold)로 나눈다 (보통은 4~5개). K개의 모델을 각각 만들어 K-1개의 분할에서 훈련하고 나머지 분할에서 평가하는 방법이다. **모델의 검증 점수**는 **K개의 검증 점수 평균**이 된다. 

코드를 보면 이해하기 쉽다. 

1. 100 에포크로 테스트해보기 

In [ ]:
#k-겹 검증하기 

import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0)
    # Evaluate the model on the validation data
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

In [ ]:
#n=100의 결과 
all_scores

In [ ]:
np.mean(all_scores)

검증 세트가 다르므로 확실히 검증 점수가 2.1에서 2.9까지 변화가 크다. 
평균(2.4)이 가각의 점수보다 훨씬 **신뢰할 만하다**. <- K-겹 교차 검증의 핵심. 이 예시에서는 평균적으로 **3000달러** 정도 차이가 난다. 주택 가격의 범위가 1만 달러에서 5만 달러 사이인 것을 감안하면 비교적 큰 값이다. 

2. 500 에포크로 테스트 해보기

신경망을 더 오래 500 에포크 동안 훈련해보자. <br>
각 에포크마다 모델이 얼마나 **개선**되는지 기록하기 위해 **훈련 루프**를 조금 수정해서 에포크의 검증 점수를 **로그**에 저장하겠다. 

In [ ]:
#책에 없음
#메모리 정리
from keras import backend as K

# Some memory clean-up
K.clear_session()

In [ ]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_model()
    # 모델 훈련(in silent mode, verbose=0)이므로 훈련과정이 출력되지 않음
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['val_mean_absolute_error']
    all_mae_histories.append(mae_history)

그런 다음, 모든 분할(폴드)에 대해 에포크의 **MAE 점수 평균**을 계산한다. 

In [ ]:
#K-겹 검증 점수 평균 기록하기 
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
#검증 점수 그래프/그림으로 나타내기 
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

* 이 그래프는 범위가 크고 변동이 심해서 가독성이 떨어진다. 다음과 같이 개선해보자 

- 곡선의 다른 부분과 스케일이 많이 다른 첫 10개의 데이터 포인트를 제외한다. 
- 부드러운 곡선을 얻기 위해 각 포인트를 이전 포인트의 **지수 이동 평균 (exponential moving average)** 로 대체한다. 지수 이동 평균은 시계열 데이터를 부드럽게 만들어주는 기법 중 하나이며 이전에 계산된 이동 평균에 factor를 곱하고 현재 포인트에 (1-factor)를 곱해 합산한 것을 가리킨다. 

In [ ]:
#처음 10개의 데이터 포인트를 제외한 에포크별 검증 MAE 그리기 
def smooth_curve(points, factor=0.9):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

이 그래프를 보면 검증 MAE가 80번째 에포크 이후에 줄어드는 것이 멈추었다. 이 지점 이후로는 **과대 적합(overfitting)** 이 시작된다. 

모델의 여러 매개변수에 대한 튜닝이 끝나면 (에포크수 뿐만 아니라 은닉 층의 크기도 조절 가능함) 모든 훈련 데이터를 사용하고 최상의 매개변수로 최종 실전에 투입될 모델을 훈련시킨다. 그 다음, 테스트 데이터로 성능을 확인한다. 


In [ ]:
#최종 모델 훈련하기 

# Get a fresh, compiled model.
model = build_model()

In [ ]:
# 전체 데이터로 훈련시킨다. 
model.fit(train_data, train_targets,
          epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

최종 결과는 다음과 같다. 

In [ ]:
test_mae_score

아직 2.675달러정도 차이가 나는 것을 볼 수 있다. 

### 3.6.5. 정리
- 회귀(regression)은 분류(classification)와는 다른 손실함수 사용. 여기서는 평균 제곱 오차 (MSE)를 사용했으며 회귀에서 자주 사용됨. 
- 회귀에서 사용되는 평가 지표도 분류와 다름. 정확도 개념은 없음. 일반적인 지표는 평균 절대 오차 (MAE)이다. 
- 입력 데이터의 특성의 범위가 다를 경우 전처리 단계에서 정규화 (regularization)가 필요함
- 가용한 데이터가 적다면 K-겹 검증을 사용하는 것이 신뢰할 수 있는 모델 평가 방법이다. 
- 가용한 훈련 데이터가 적다면 은닉 층의 수를 줄인 모델 (일반적으로 1~2개)을 써야 과대 적합을 피할 수 있다. 